# Zillow Feature Engineering
## Import Packages

In [ ]:
import numpy as np
import pandas as pd 
from scipy import stats
import warnings
import time
import numba as nb

# display set up
%precision 4
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 3)

## Functions Definition

In [ ]:
# Timer decorator
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('----------------------')
            print('Function %r takes %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [ ]:
# Functions for memory reduction
@nb.jit()
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return(df)

## Load in Dataset

In [31]:
%%time
# Load in target dataa
df_2016 = pd.read_csv('train_2016_v2.csv')
df_2017 = pd.read_csv('train_2017.csv')

# Load in properties data
prop_2016 = pd.read_csv('properties_2016.csv')
prop_2017 = pd.read_csv('properties_2017.csv')
assert len(prop_2016) == len(prop_2017)
print(f"Number of properties: {len(prop_2016)}".format(len(prop_2016)))
print(f"Number of property features: {len(prop_2016.columns)-1}")

# Reduce Size
prop_2016 = reduce_mem_usage(prop_2016)
prop_2017 = reduce_mem_usage(prop_2017)
print(f"Rows for prop 2016: {len(prop_2016)}")
print(f"Rows for prop 2017: {len(prop_2017)}")

KeyboardInterrupt: 

---
## Feature Engineering
From what we have learned in the Exploratory Data Analysis part, we can do the following feature engineering process. Some Engineering process is from my personal exploratory analysis, and some others are from some fantastic and insightful kaggle notebooks learned from other machine learning experts.

### Define useless columns

In [32]:
# These columns will be drop later.
drop_list = ['parcelid','architecturalstyletypeid','buildingclasstypeid','calculatedbathnbr','decktypeid',
 'finishedfloor1squarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13','finishedsquarefeet15',
 'finishedsquarefeet6', 'fullbathcnt', 'regionidcounty', 
 'regionidneighborhood', 'typeconstructiontypeid', 'taxvaluedollarcnt', 'assessmentyear',
 'censustractandblock']

### Extract Datetime

In [33]:
def extract_datetime(df):
    dt = pd.to_datetime(df.transactiondate).dt
    df['year'] = (dt.year - 2016).astype(int)
    df['month'] = (dt.month).astype(int)
    df['quarter'] = (dt.quarter).astype(int)
    df.drop(['transactiondate'], axis=1, inplace=True)
    return(df)

In [34]:
df_2016 = extract_datetime(df_2016)
df_2017 = extract_datetime(df_2017)

### Missing Pattern

Inspired by the post below that talks about the null pattern. There are indeed lots of null values in this dataset. We can simply create some vectors to recognize those null patterns, which may be helpful to know the overall structure of the dataset.
https://www.kaggle.com/c/ieee-fraud-detection/discussion/108727

In [ ]:
from sklearn.decomposition import PCA

def get_null_pattern(df):
    null_pattern = df.isnull().astype(int)
    pca = PCA(n_components=10)
    columns = ['NAPattern1','NAPattern2','NAPattern3','NAPattern4','NAPattern5','NAPattern6','NAPattern7',
              'NAPattern8','NAPattern9','NAPattern10']
    null_embedding = pd.DataFrame(pca.fit_transform(null_pattern), columns=columns)
    df = pd.concat([df,null_embedding],axis=1)
    return(df)

prop_2016 = get_null_pattern(prop_2016)
prop_2017 = get_null_pattern(prop_2017)

### Categorical Features

There are lots of missing values in the categorical columns. Let's encode them into one of the unseen value so that we can afterwards groupby those categorical columns.

In [ ]:
def impute_categorical(df, columns):
    for col in columns:
        df.loc[df[col].isnull(),col] = -1
        df[col] = df[col].astype('category')
    return(df)

impute_columns = ['regionidzip','propertyzoningdesc','propertycountylandusecode','regionidcounty','regionidcity',
                 'airconditioningtypeid','buildingqualitytypeid','fips','heatingorsystemtypeid']
prop_2016 = impute_categorical(prop_2016, impute_columns)
prop_2017 = impute_categorical(prop_2017, impute_columns)

Now let's use latitude and longitude data to get some insights from the geo-location information.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

In [ ]:
geo_list = ['latitude', 'longitude']

for col in geo_list:
    prop_2016[col] = prop_2016[col].apply(lambda x: float(x))/10e6
    prop_2016[col] = prop_2016[col].fillna(stats.mode(prop_2016[col])[0][0])
    
for col in geo_list:
    prop_2017[col] = prop_2017[col].apply(lambda x: float(x))/10e6
    prop_2017[col] = prop_2017[col].fillna(stats.mode(prop_2017[col])[0][0])
    
@timeit
def get_optimal_geocluster(prop_2016, prop_2017):
    K = range(2,16)
    for df in [prop_2016, prop_2017]:
        mapping_dav = {} 

        print(f'Start Fitting.')
        for k in K: 
            #Building and fitting the model 
            kmeanModel = KMeans(n_clusters=k)
            pred = kmeanModel.fit_predict(prop_2016[geo_list])     

            dav = davies_bouldin_score(prop_2016[geo_list], pred)
            print(dav)
            mapping_dav[k] = dav

            print(f"Iteration {k} finished. The davies bouldin score for {k} cluster is: {np.round(dav,4)}")
        
        print(f'The best 5 cluster number is '+ str(sorted(mapping_dav, key=mapping_dav.get, reverse=True)[:5]))
        print('---------------------------------------')

Since the lower the davie-bouldin index the better, we know that **k = 10** is the best number of clusters!

In [ ]:
prop_2016['geo_cluster'] = KMeans(n_clusters=10).fit_predict(prop_2016[geo_list])
prop_2017['geo_cluster'] = KMeans(n_clusters=10).fit_predict(prop_2017[geo_list])

In [ ]:
# Other Insightful Transformation for geographical features 

## The location might be one important issue. 
## However, we don't know how the latitutde and longitude value is transformed here.
## Let's do some basic transformation
def get_rotated_coord(df):
    ## Rotated Coordinates
    df['coord_1'] = df['latitude'] + df['longitude']
    df['coord_2'] = df['latitude'] - df['longitude']
    df['coord_3'] = df['latitude'] + 2 * df['longitude']
    df['coord_4'] = df['latitude'] - 2 * df['longitude']
    return(df)

prop_2016 = get_rotated_coord(prop_2016)    
prop_2017 = get_rotated_coord(prop_2017) 
drop_list.extend(geo_list) # Drop those latitude and longitude data out.


## Why NYC and Silicon Valley is very expensive? Because the demand is high!
## Demand and supply is one important issue for house pricing.
## Since the regioneighborhood has lots of missing value, here we use regionzip to define neighborhood instead 
def get_neighborhood(df):
    df['regionidcity_groupcnt'] = df['regionidcity'].map(df['regionidcity'].value_counts())
    return(df)

prop_2016 = get_neighborhood(prop_2016)    
prop_2017 = get_neighborhood(prop_2017) 

As we have learned that there are features for bathroom and bedroom. We also have features for the total area for the entire house -- from there we can calculate the **average size of room**. 

There is a bathroom and bedroom feature. Why not construct a roomcount feature from that? 

In [ ]:
prop_2016[['roomcnt','bedroomcnt','bathroomcnt']].sample(2)

In [ ]:
def get_aggregate_room(df):
    ## Roomcount for each house
    df['derived_room_cnt'] = df['bedroomcnt'] + df['bathroomcnt']

    ## Why there's a difference? Maybe here's how the error comes from
    df['diff_derived_roomcnt'] = df['derived_room_cnt'] - df['roomcnt']

    ## Average Size for each room for house
    mask = (df.derived_room_cnt >= 1)
    df.loc[mask, 'avg_room_size'] = df.loc[mask, 'calculatedfinishedsquarefeet'] / df.loc[mask, 'derived_room_cnt']

    ## Average Size for each garage
    mask = (df.garagecarcnt >= 1)
    df.loc[mask,'avg_garage_size'] = df.loc[mask, 'garagetotalsqft'] / df.loc[mask,'garagecarcnt']

    ## Average Tax Rate
    df['property_tax_per_sqft'] = df['taxamount'] / df['calculatedfinishedsquarefeet']
    return(df)

prop_2016 = get_aggregate_room(prop_2016)
prop_2017 = get_aggregate_room(prop_2017)

    
# Average Size per room across each category
def add_aggregate_across_categories(df, groupby_cols, aggregate_col):
    dataframe = df.copy(deep=True)   
    df = df[df[aggregate_col].notna()]
    for col in groupby_cols:
        temp = df.groupby(col)[aggregate_col].agg([np.mean])
        temp.columns = [f'{aggregate_col}_across_{col}']
        try:
            dataframe = dataframe.merge(how='left', right=temp, on=f'{col}')
        except:
            temp[f'{col}'] = temp[f'{col}'].astype(np.str)
            dataframe = dataframe.merge(how='left', right=temp, on=f'{col}')
    
    for col in groupby_cols:
        mean = dataframe[f'{aggregate_col}_across_{col}']
        diff = dataframe[col] - mean
        
        dataframe[f'{aggregate_col}_across_{col}_diff'] = diff
        dataframe[f'{aggregate_col}_across_{col}_percent'] = diff / mean
    
    return(dataframe)

groupby_cols = ['airconditioningtypeid','buildingqualitytypeid','heatingorsystemtypeid',
                'propertycountylandusecode','propertylandusetypeid','propertyzoningdesc',
                'rawcensustractandblock', 'regionidcity','regionidneighborhood','regionidzip',
                'yearbuilt','assessmentyear','taxdelinquencyyear','censustractandblock','geo_cluster']

aggregate_col = 'avg_room_size'

prop_2016 = add_aggregate_across_categories(prop_2016, groupby_cols, aggregate_col)
prop_2017 = add_aggregate_across_categories(prop_2017, groupby_cols, aggregate_col)

In [ ]:
def add_aggregate_across_geo(df, groupby_col, aggregate_cols):
    dataframe = df.copy(deep=True)
    new_columns = []  # New feature columns added to the DataFrame

    for col in aggregate_cols:
        temp = df.groupby(groupby_col, as_index=False)[col].agg([np.mean])
        temp.columns = [f'{col}_across_{groupby_col}']
        new_columns += list(temp.columns)
        try:
            dataframe = dataframe.merge(how='left', right=temp, on=f'{groupby_col}')
        except:
            temp[f'{col}'] = temp[f'{col}'].astype(np.str)
            dataframe = dataframe.merge(how='left', right=temp, on=f'{groupby_col}')
    
     for col in groupby_cols:
        mean = dataframe[f'{col}_across_{groupby_col}']
        diff = dataframe[col] - mean
        
        dataframe[f'{col}_across_{groupby_col}_diff'] = diff
        if col != 'year_built':
            dataframe[f'{col}_across_{groupby_col}_percent'] = diff / mean
    
    return(dataframe)

groupby_col = 'regionidzip'
aggregate_cols = ['lotsizesquarefeet', 'calculatedfinishedsquarefeet', 'yearbuilt'
            'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount', 'property_tax_per_sqft']

prop_2016 = add_aggregate_across_geo(prop_2016, groupby_col, aggregate_cols)
prop_2017 = add_aggregate_across_geo(prop_2017, groupby_col, aggregate_cols)

In [ ]:
prop_2016[['censustractandblock','rawcensustractandblock']].sample(2)

In [ ]:
def get_census_alignment(df):
    return(((df['censustractandblock'] / 10e5) > df['rawcensustractandblock']).apply(int))

prop_2016 = get_census_alignment(prop_2016)
prop_2017 = get_census_alignment(prop_2017)

## Save Features

In [41]:
%%time

# Drop unneeded columns
prop_2016 = prop_2016.drop(columns=drop_list)
prop_2017 = prop_2017.drop(columns=drop_list)

# Reduce Size
prop_2016 = reduce_mem_usage(prop_2016)
prop_2017 = reduce_mem_usage(prop_2017)

print(f"Number of properties: {len(prop_2016)}".format(len(prop_2016)))
print(f"Number of property features: {len(prop_2016.columns)-1}")

# Write feature DataFrames to csv
prop_2016.to_csv('data/prop_2016.csv', index=False)
prop_2017.to_csv('data/prop_2017.csv', index=False)

Mem. usage decreased to 689.20 Mb (39.9% reduction)
Mem. usage decreased to 683.50 Mb (40.4% reduction)
Number of properties: 2985217
Number of property features: 91
CPU times: user 5min 33s, sys: 4.49 s, total: 5min 37s
Wall time: 5min 40s


## Training Data Preparation

In [42]:
# Merge Dataset
df_prop_2016 = pd.merge(df_2016,prop_2016,how='left',on='parcelid')
df_prop_2017 = pd.merge(df_2017,prop_2017,how='left',on='parcelid')

# Combine the 2016 and 2017 training sets
train = pd.concat([df_prop_2016, df_prop_2017], axis=0, ignore_index=True)
print(f"Combined training set size: {len(train)}")

Combined training set size: 167888


## Save Data

In [43]:
%%time
# Reduce Size
train = reduce_mem_usage(train)

# Write training DataFrame to CSV
train.to_csv('data/train.csv', index=False)

Mem. usage decreased to 38.60 Mb (16.3% reduction)
CPU times: user 9.57 s, sys: 104 ms, total: 9.67 s
Wall time: 9.7 s
